In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [1]:
data = pd.read_csv('pima-indians-diabetes.csv')
print(data.shape)
data.head()

<IPython.core.display.Javascript object>

(768, 9)


,Preg,Plas,Pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
X = data.drop(columns='class', axis=1)
y = data['class']

### **Cross validation**

In [8]:
k = 10

kfold = KFold(n_splits=k, shuffle=True, random_state=42)
model = LogisticRegression(max_iter=500)
results = cross_val_score(model, X, y, cv=kfold)
print(results)
print('Accuracy: %.3f%% (%.3f%%)' % (results.mean()*100, results.std()*100))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[0.7012987  0.80519481 0.72727273 0.84415584 0.83116883 0.67532468
 0.85714286 0.77922078 0.69736842 0.78947368]
Accuracy: 77.076% (6.270%)


Our model perform on average 77%. And could vary between 70.806% and 83.346%. 